This notebook test the NN from scratch implementation of our group on two dataset. 
- We test on the Student Academic Success and 
- Congressional Voting 

In [142]:
def build_nn(model_type, input_dim, output_dim):
    nn = NeuralNetwork()

    if model_type == "1_relu":
        nn.add_layer(input_dim, 6, "relu")
        nn.add_layer(6, output_dim, "softmax")

    elif model_type == "1_sigmoid":
        nn.add_layer(input_dim, 6, "sigmoid")
        nn.add_layer(6, output_dim, "softmax")

    elif model_type == "4_relu":
        nn.add_layer(input_dim, 32, "relu")
        nn.add_layer(32, 16, "relu")
        nn.add_layer(16, 8, "relu")
        nn.add_layer(8, 4, "relu")
        nn.add_layer(4, output_dim, "softmax")

    elif model_type == "4_sigmoid":
        nn.add_layer(input_dim, 32, "sigmoid")
        nn.add_layer(32, 16, "sigmoid")
        nn.add_layer(16, 8, "sigmoid")
        nn.add_layer(8, 4, "sigmoid")
        nn.add_layer(4, output_dim, "softmax")

    return nn

In [143]:
def train_model(model_name, X_train, y_train, X_test, y_test, epochs=100, lr=0.05):
    nn = build_nn(model_name, input_dim=X_train.shape[1], output_dim=len(np.unique(y_train)))

    for epoch in range(epochs):
        y_pred = nn.forward(X_train)
        loss = LossFunction.categorical_cross_entropy(y_pred, y_train)
        grad = LossFunction.categorical_cross_entropy_derivative(y_pred, y_train)
        nn.backward(grad, learning_rate=lr)

        if epoch % 10 == 0 or epoch == epochs - 1:
            acc = np.mean(np.argmax(y_pred, axis=1) == y_train)
            print(f"[{model_name.upper()}] Epoch {epoch}: Loss = {loss:.4f}, Accuracy = {acc:.4f}")

    y_test_pred = nn.forward(X_test)
    acc_test = np.mean(np.argmax(y_test_pred, axis=1) == y_test)
    print(f"[{model_name.upper()}] Final Test Accuracy: {acc_test:.4f}")

## Student Dropout Dataset

In [145]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from nn_mir0 import NeuralNetwork, LossFunction

In [146]:
students_success_data_path = 'student_droupout_data.csv'
df = pd.read_csv(students_success_data_path, sep=';')
label_map = {'Dropout': 0, 'Enrolled': 1, 'Graduate': 2}

In [147]:
X = df.drop("Target", axis=1).values
y = np.array([label_map[label] for label in df["Target"]])

In [148]:
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [149]:
for model in ["1_relu", "1_sigmoid", "4_relu", "4_sigmoid"]:
    train_model(model, X_train, y_train, X_test, y_test)

[1_RELU] Epoch 0: Loss = 1.7137, Accuracy = 0.5565
[1_RELU] Epoch 10: Loss = 1.0268, Accuracy = 0.5284
[1_RELU] Epoch 20: Loss = 1.0010, Accuracy = 0.5484
[1_RELU] Epoch 30: Loss = 0.9804, Accuracy = 0.5672
[1_RELU] Epoch 40: Loss = 0.9630, Accuracy = 0.5801
[1_RELU] Epoch 50: Loss = 0.9477, Accuracy = 0.5917
[1_RELU] Epoch 60: Loss = 0.9338, Accuracy = 0.6027
[1_RELU] Epoch 70: Loss = 0.9211, Accuracy = 0.6079
[1_RELU] Epoch 80: Loss = 0.9093, Accuracy = 0.6176
[1_RELU] Epoch 90: Loss = 0.8980, Accuracy = 0.6224
[1_RELU] Epoch 99: Loss = 0.8884, Accuracy = 0.6311
[1_RELU] Final Test Accuracy: 0.6137
[1_SIGMOID] Epoch 0: Loss = 1.0648, Accuracy = 0.4903
[1_SIGMOID] Epoch 10: Loss = 1.0283, Accuracy = 0.4919
[1_SIGMOID] Epoch 20: Loss = 1.0052, Accuracy = 0.4955
[1_SIGMOID] Epoch 30: Loss = 0.9903, Accuracy = 0.5000
[1_SIGMOID] Epoch 40: Loss = 0.9804, Accuracy = 0.5048
[1_SIGMOID] Epoch 50: Loss = 0.9731, Accuracy = 0.5271
[1_SIGMOID] Epoch 60: Loss = 0.9673, Accuracy = 0.6072
[1_SIGMO

## Congressional Voting Dataset

In [151]:
from sklearn.impute import SimpleImputer

congressional_voting_training_data_path = 'CongressionalVotingID.csv'
df = pd.read_csv(congressional_voting_training_data_path)
df.drop(df[np.isin(df['ID'], [184, 249])].index, inplace=True)
df.drop(columns=['ID'], inplace=True)

In [152]:
label_map = {'democrat': 0, 'republican': 1}
y = np.array([label_map[val] for val in df["class"]])

In [153]:
X = df.drop("class", axis=1).values
X = np.where(X == 'unknown', np.nan, X)
X = np.where(X == 'y', 1, X)
X = np.where(X == 'n', 0, X)

In [154]:
X = SimpleImputer(strategy='most_frequent').fit_transform(X).astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [155]:
for model in ["1_relu", "1_sigmoid", "4_relu", "4_sigmoid"]:
    train_model(model, X_train, y_train, X_test, y_test)

[1_RELU] Epoch 0: Loss = 0.9459, Accuracy = 0.4305
[1_RELU] Epoch 10: Loss = 0.9929, Accuracy = 0.4702
[1_RELU] Epoch 20: Loss = 0.7297, Accuracy = 0.6821
[1_RELU] Epoch 30: Loss = 0.3674, Accuracy = 0.8675
[1_RELU] Epoch 40: Loss = 0.2949, Accuracy = 0.8675
[1_RELU] Epoch 50: Loss = 0.2763, Accuracy = 0.8675
[1_RELU] Epoch 60: Loss = 0.2605, Accuracy = 0.8742
[1_RELU] Epoch 70: Loss = 0.2468, Accuracy = 0.8874
[1_RELU] Epoch 80: Loss = 0.2346, Accuracy = 0.8874
[1_RELU] Epoch 90: Loss = 0.2236, Accuracy = 0.9007
[1_RELU] Epoch 99: Loss = 0.2146, Accuracy = 0.9139
[1_RELU] Final Test Accuracy: 0.9692
[1_SIGMOID] Epoch 0: Loss = 0.7066, Accuracy = 0.4172
[1_SIGMOID] Epoch 10: Loss = 0.6798, Accuracy = 0.5828
[1_SIGMOID] Epoch 20: Loss = 0.6792, Accuracy = 0.5828
[1_SIGMOID] Epoch 30: Loss = 0.6791, Accuracy = 0.5828
[1_SIGMOID] Epoch 40: Loss = 0.6791, Accuracy = 0.5828
[1_SIGMOID] Epoch 50: Loss = 0.6790, Accuracy = 0.5828
[1_SIGMOID] Epoch 60: Loss = 0.6790, Accuracy = 0.5828
[1_SIGMO